In [1083]:
import os
import glob
import pymssql
import pandas as pd
import xml.etree.ElementTree as ET

In [1084]:
path = '/home/boyeon/Desktop/workspace/code/annotation_program/xml_test'
xml_list = glob.glob(path + '/*.xml')

# db 연동, sku_train table 가져오기

In [1085]:
server = '192.168.0.85:11433'
user = 'interminds'
password = 'ntflow'
database = 'cspace_test'
port = 11433

conn = pymssql.connect(server, user, password, database, charset='CP949')

SQL_QUERY = """
SELECT * from sku_train st ;
"""

cursor = conn.cursor()
cursor.execute(SQL_QUERY)
records = cursor.fetchall()
conn.close()

db_df = pd.DataFrame(records, 
columns=['sku_id','product_name','manufacturer','ref_image_file','used_for_tr','sku_barcode','category','class_info_id','barcode','excel_no'
,'brand_nm'])

db_df.tail()

,sku_id,product_name,manufacturer,ref_image_file,used_for_tr,sku_barcode,category,class_info_id,barcode,excel_no,brand_nm
723,729,오늘좋은 스위트&솔티 팝콘,기타,알수 없음,None,8809027550435,스낵,0,8809027550435,738,기타(스낵)
724,730,오늘좋은 커피땅콩맛 팝콘,기타,알수 없음,None,8809027550459,스낵,0,8809027550459,739,기타(스낵)
725,731,오늘좋은 카라멜맛 팝콘,기타,알수 없음,None,8809027550411,스낵,0,8809027550411,740,기타(스낵)
726,732,오늘좋은 버터갈릭&어니언 팝콘,기타,알수 없음,None,8809027550398,스낵,0,8809027550398,741,기타(스낵)
727,733,오늘좋은 오리지널&카라멜맛 팝콘,기타,알수 없음,None,8809027550404,스낵,0,8809027550404,742,기타(스낵)


# annotation, db table matching

In [1086]:
dic = {}
for xml in xml_list:
    tree = ET.parse(xml)
    root = tree.getroot()
    for x in root.findall('object'):
        name = (x.find('name').text)

        if name not in dic:
            dic[name]=0
        dic[name]+=1

In [1090]:
sku_id = dic.keys() 
patch_count= dic.values()
annotation_df = pd.DataFrame({'excel_no': sku_id, 'patch count':patch_count})
annotation_df = annotation_df.sort_values(by='patch count', ascending=False)
# annotation_df['excel_no'] = annotation_df['excel_no'].astype(int)
annotation_df

,excel_no,patch count
12,150,779
71,197,443
321,102,280
70,186,215
112,59,204
...,...,...
622,327,1
361,711,1
629,675,1
624,693,1


In [1091]:
merged_df = annotation_df.merge(db_df, left_on = 'excel_no', right_on = 'excel_no')

In [1092]:
merged_df.head()

,excel_no,patch count,sku_id,product_name,manufacturer,ref_image_file,used_for_tr,sku_barcode,category,class_info_id,barcode,brand_nm
0,150,779,154,알수없음,알수없음,알수 없음,None,0,알수없음,0,,알수없음
1,197,443,200,해태 홈런볼 초코,해태,알수 없음,None,8801019306495,비스켓,0,8801019306495,홈런볼
2,102,280,106,오리온 오징어 땅콩(오리지널),오리온,알수 없음,None,8801117761004,스낵,0,8801117761004,오징어땅콩
3,186,215,189,롯데 꼬깔콘 고소한맛,롯데,알수 없음,None,8801062010578,스낵,0,8801062010578,꼬깔콘
4,59,204,63,"오리온 초코파이(12개입,18개입)",오리온,알수 없음,None,8801117534912,파이,0,8801117534912,초코파이


In [1093]:
merged_df = merged_df[['sku_id','excel_no', 'patch count', 'product_name', 'manufacturer', 'category', 'brand_nm', 'barcode']]

In [1103]:
for i in range(len(merged_df)):
    if merged_df.loc[i, 'patch count'] < 30:
        merged_df.loc[i, 'modified_product_name'] = str(merged_df.loc[i, 'manufacturer']) + ' ' + str(merged_df.loc[i, 'brand_nm']) + ' 기타'
    else:
        pass
merged_df.tail()

,sku_id,excel_no,patch count,product_name,manufacturer,category,brand_nm,barcode,new_name,modified_product_name
692,324,327,1,크라운 와글와글 스낵 파티 5팩 286G,크라운,스낵,기타(스낵),8801111949439,크라운 기타(스낵) 기타,크라운 기타(스낵) 기타
693,702,711,1,해태 넷플릭스 나쵸 치즈,해태,스낵,None,8801728107659,해태 None 기타,해태 None 기타
694,666,675,1,오늘좋은 강냉이,기타,스낵,기타(스낵),8809020659463,기타 기타(스낵) 기타,기타 기타(스낵) 기타
695,684,693,1,청우 쫀득초코칩 초코바나나,기타,비스켓,쫀득초코칩,8801204205633,기타 쫀득초코칩 기타,기타 쫀득초코칩 기타
696,704,713,1,HBAF 당근케이크아몬드 190g,기타,,None,8809617945214,기타 None 기타,기타 None 기타


In [1106]:
new_label = [(f'etc_{i-1}', v) for i, v in enumerate(merged_df['new_name'].unique()) if type(v) != float]
a = [i[0] for i in new_label]
b = [i[1] for i in new_label]
df2 = pd.DataFrame({'modified_excel_no':a, 'modified_product_name':b})
df2.head(10)

,modified_excel_no,modified_product_name
0,etc_0,기타 기타(스낵) 기타
1,etc_1,해태 구운감자 기타
2,etc_2,오리온 예감 기타
3,etc_3,해태 포키 기타
4,etc_4,오리온 도도한나쵸 기타
5,etc_5,오리온 고소미 기타
6,etc_6,롯데 씨리얼 기타
7,etc_7,롯데 하비스트 기타
8,etc_8,청우 그랑쉘 기타
9,etc_9,농심 프링글스 기타


In [1112]:
unique_modified_labels = merged_df['modified_product_name'].unique()

In [1113]:
modified_labels_df = pd.DataFrame({'modified_excel_no':[f'etc_{i-1}' for i in range(len(unique_modified_labels)) if type(unique_modified_labels[i]) != float], 
                                       'modified_product_name' :[i for i in unique_modified_labels if type(i) != float]})

In [1114]:
df = pd.merge(merged_df, modified_labels_df, how='left', on='modified_product_name')

In [1118]:
df.loc[df['modified_excel_no'].notna(), ['excel_no', 'modified_excel_no']]

,excel_no,modified_excel_no
240,462,etc_0
241,243,etc_1
242,244,etc_2
243,107,etc_3
244,308,etc_4
...,...,...
692,327,etc_109
693,711,etc_69
694,675,etc_0
695,693,etc_110


In [1101]:
for i in range(len(df)):
#     print(df.loc[i, 'label_name'])
    if type(df.loc[i, 'label_name']) == float:
        df.loc[i, 'label_name'] =  df.loc[i, 'excel_no']
    if type(df.loc[i, 'new_name']) == float:
        df.loc[i, 'new_name'] = df.loc[i, 'product_name']

In [1100]:
df

,sku_id,excel_no,patch count,product_name,manufacturer,category,brand_nm,barcode,new_name,label_name
0,154,150,779,알수없음,알수없음,알수없음,알수없음,,알수없음,150
1,200,197,443,해태 홈런볼 초코,해태,비스켓,홈런볼,8801019306495,해태 홈런볼 초코,197
2,106,102,280,오리온 오징어 땅콩(오리지널),오리온,스낵,오징어땅콩,8801117761004,오리온 오징어 땅콩(오리지널),102
3,189,186,215,롯데 꼬깔콘 고소한맛,롯데,스낵,꼬깔콘,8801062010578,롯데 꼬깔콘 고소한맛,186
4,63,59,204,"오리온 초코파이(12개입,18개입)",오리온,파이,초코파이,8801117534912,"오리온 초코파이(12개입,18개입)",59
...,...,...,...,...,...,...,...,...,...,...
692,324,327,1,크라운 와글와글 스낵 파티 5팩 286G,크라운,스낵,기타(스낵),8801111949439,크라운 기타(스낵) 기타,etc_109
693,702,711,1,해태 넷플릭스 나쵸 치즈,해태,스낵,None,8801728107659,해태 None 기타,etc_69
694,666,675,1,오늘좋은 강냉이,기타,스낵,기타(스낵),8809020659463,기타 기타(스낵) 기타,etc_0
695,684,693,1,청우 쫀득초코칩 초코바나나,기타,비스켓,쫀득초코칩,8801204205633,기타 쫀득초코칩 기타,etc_110


In [1046]:
df.drop_duplicates(['new_name', 'label_name'], inplace=True)

In [996]:
df

,sku_id,excel_no,patch count,product_name,manufacturer,category,brand_nm,barcode,new_name,label_name
0,154,150,779,알수없음,알수없음,알수없음,알수없음,,알수없음,150
1,200,197,443,해태 홈런볼 초코,해태,비스켓,홈런볼,8801019306495,해태 홈런볼 초코,197
2,106,102,280,오리온 오징어 땅콩(오리지널),오리온,스낵,오징어땅콩,8801117761004,오리온 오징어 땅콩(오리지널),102
3,189,186,215,롯데 꼬깔콘 고소한맛,롯데,스낵,꼬깔콘,8801062010578,롯데 꼬깔콘 고소한맛,186
4,63,59,204,"오리온 초코파이(12개입,18개입)",오리온,파이,초코파이,8801117534912,"오리온 초코파이(12개입,18개입)",59
...,...,...,...,...,...,...,...,...,...,...
575,179,175,3,롯데 The쌀로 바삭한 핫칠리맛,롯데,스낵,쌀로별,8801062860418,롯데 쌀로별 기타,etc_104
608,423,426,2,농심 맛있는 이야기 720g (24입,농심,,None,,농심 None 기타,etc_105
680,597,605,1,오리온 닥터유 에너지바 박스,오리온,,None,,오리온 None 기타,etc_106
692,324,327,1,크라운 와글와글 스낵 파티 5팩 286G,크라운,,None,8801111949439,크라운 None 기타,etc_107


In [1082]:
excel_path = 'class_info.xlsx'
data = pd.read_excel(excel_path, index_col=0).sort_values('sku_id')
data

,sku_id,excel_no,patch count,product_name,manufacturer,category,brand_nm,barcode
140,6.0,2,52,오리온 예감 오리지널 64g,오리온,비스켓,예감,8.801117e+12
209,7.0,3,37,오리온 예감 치즈그라탕맛 64g,오리온,비스켓,예감,8.801117e+12
231,8.0,4,31,오리온 예감 볶은양파 64g,오리온,비스켓,예감,8.801117e+12
203,9.0,5,37,롯데 롯샌 파인애플 105g,롯데,비스켓,롯데샌드,8.801063e+12
61,10.0,6,84,롯데 빼빼로 오리지널 54g,롯데,비스켓,빼빼로,8.801062e+12
...,...,...,...,...,...,...,...,...
346,NaN,etc_106,3,롯데 쌀로별 기타,롯데,스낵,쌀로별,8.801063e+12
347,NaN,etc_107,2,농심 기타(스낵) 기타,농심,스낵,기타(스낵),NaN
348,NaN,etc_108,1,오리온 None 기타,오리온,NaN,NaN,NaN
349,NaN,etc_109,1,크라운 기타(스낵) 기타,크라운,스낵,기타(스낵),8.801112e+12


In [1078]:
dic = {}

In [1079]:
for i, v in data.iterrows():
    id = v['excel_no']
    name = v['product_name']
#     dic.append({'id':id, 'name':name})
    if id not in dic.keys():
        dic[id] = name
dic

{150: '알수없음',
 197: '해태 홈런볼 초코',
 102: '오리온 오징어 땅콩(오리지널)',
 186: '롯데 꼬깔콘 고소한맛',
 59: '오리온 초코파이(12개입,18개입)',
 185: '롯데 꼬깔콘 군옥수수맛',
 57: '오리온 초코송이 36g',
 158: '해태 맛동산',
 113: '해태 허니버터칩',
 22: '오리온 마켓오 브라우니',
 303: '해태 홈런볼커스타드크림',
 71: '롯데 제로 쿠앤크샌드',
 137: '크라운 콘칩',
 10: '해태 에이스 121g',
 110: '오리온 포카칩 오리지널 ',
 68: '롯데 제로 카카오케이크 171g(12봉입)',
 41: '크라운 빅파이',
 173: '롯데 치토스 매콤달콤한맛',
 118: '농심 새우깡',
 67: '롯데 카스타드 오리지날 230g',
 157: '크라운 죠리퐁',
 45: '롯데 몽쉘 생크림케이크 카카오(6봉입, 12봉입)',
 249: '롯데 마가렛트 오리지널',
 191: '오리온 썬칩 핫스파이시맛',
 50: '오리온 참붕어빵(8개입,18개입)',
 46: '롯데 몽쉘 생크림케이크 오리지널408g(6봉입, 12봉입)',
 111: '오리온 포카칩 어니언맛 ',
 65: '크라운 뽀또 치즈타르트 322g',
 61: '크라운 버터와플 12개입',
 83: '롯데 초코파이 480g(12개입)',
 229: '농심 포테토칩 오리지널',
 174: '롯데 치토스 스모키바베큐맛',
 163: '오리온 땅콩 강정',
 32: '크라운 쵸코하임',
 26: '크라운 참크래커 280g',
 198: '크라운 쿠크다스 비엔나 커피 ',
 154: '해태 구운양파',
 40: '해태 후렌치 파이 딸기',
 128: '오리온 꼬북칩 콘스프',
 199: '크라운 쿠크다스 화이트 토르테 ',
 74: '롯데 빈츠 204g',
 187: '롯데 꼬깔콘 매콤달콤한맛',
 81: '오리온 후레쉬베리',
 169: '해태 오사쯔',
 35: '해태 오예스 360g(12입',


In [1074]:
dic

[{'id': 150, 'name': '알수없음'},
 {'id': 197, 'name': '해태 홈런볼 초코'},
 {'id': 102, 'name': '오리온 오징어 땅콩(오리지널)'},
 {'id': 186, 'name': '롯데 꼬깔콘 고소한맛'},
 {'id': 59, 'name': '오리온 초코파이(12개입,18개입)'},
 {'id': 185, 'name': '롯데 꼬깔콘 군옥수수맛'},
 {'id': 57, 'name': '오리온 초코송이 36g'},
 {'id': 158, 'name': '해태 맛동산'},
 {'id': 113, 'name': '해태 허니버터칩'},
 {'id': 22, 'name': '오리온 마켓오 브라우니'},
 {'id': 303, 'name': '해태 홈런볼커스타드크림'},
 {'id': 71, 'name': '롯데 제로 쿠앤크샌드'},
 {'id': 137, 'name': '크라운 콘칩'},
 {'id': 10, 'name': '해태 에이스 121g'},
 {'id': 110, 'name': '오리온 포카칩 오리지널 '},
 {'id': 68, 'name': '롯데 제로 카카오케이크 171g(12봉입)'},
 {'id': 41, 'name': '크라운 빅파이'},
 {'id': 173, 'name': '롯데 치토스 매콤달콤한맛'},
 {'id': 118, 'name': '농심 새우깡'},
 {'id': 67, 'name': '롯데 카스타드 오리지날 230g'},
 {'id': 157, 'name': '크라운 죠리퐁'},
 {'id': 45, 'name': '롯데 몽쉘 생크림케이크 카카오(6봉입, 12봉입)'},
 {'id': 249, 'name': '롯데 마가렛트 오리지널'},
 {'id': 191, 'name': '오리온 썬칩 핫스파이시맛'},
 {'id': 50, 'name': '오리온 참붕어빵(8개입,18개입)'},
 {'id': 46, 'name': '롯데 몽쉘 생크림케이크 오리지널408g(6봉입, 12봉입)'}